In [15]:
import numpy as np
from textable import table

In [16]:
def _one_atom(arrayn, name):
    varFirst = True
    listHKL = []
    listM = []
    for h in range(1, 10):
        for k in range(h + 1):
            for l in range(k + 1):
                fltStruct = 0
                strOut = 'build/hkl_one/{0}.dat'.format(name)
                for entry in array:
                    exponent = entry[0]*h + entry[1]*k + entry[2]*l
                    fltStruct += np.exp(-2j*np.pi*exponent)
                if abs(fltStruct) >= 0.001:
                    fltM = h**2 + k**2 + l**2
                    if varFirst:
                        fltFirst = fltM
                        varFirst = False
                    if fltM not in listM and np.sqrt(fltM / float(fltFirst)) < 6:
                        listM.append(fltM)
                        listHKL.append([h, k, l, fltM, np.sqrt(fltM / float(fltFirst)), 0])
    with open(
            strOut, 'w'
            ) as f:
        for row in listHKL:
            for entry in row:
                f.write('{0}\t'.format(entry))
            f.write('\n')

In [27]:
def _two_atoms(array, name, arrPositive, arrNegative, listNumber, listPositive, listNegative, inv):
    for idxPos, listpos in enumerate(arrPositive):
        for idxNeg, listneg in enumerate(arrNegative):
            varFirst = True
            varSkip = False
            listHKL = []
            listM = []
            for idxList, pos, neg, number in zip(range(len(listpos)), listpos, listneg, listNumber):
                listM.append([])
                if neg == 0 or pos == 0:
                    continue
                for h in range(1, 10):
                    for k in range(h + 1):
                        for l in range(k + 1):
                            fltStruct = 0
                            if inv:
                                strOut = 'build/hkl_two/{0}_{1}_{2}.dat'.format(
                                    name, listNegative[idxNeg], listPositive[idxPos]
                                    )
                                for arrNumber, arrIdx in enumerate(array):
                                    for entry in arrIdx:
                                        exponent = entry[0]*h + entry[1]*k + entry[2]*l
                                        if arrNumber == 0:
                                            fltStruct += neg * np.exp(-2j*np.pi*exponent)
                                        else:
                                            fltStruct += pos * np.exp(-2j*np.pi*exponent)
                            else:
                                strOut = 'build/hkl_two/{0}_{1}_{2}.dat'.format(
                                    name, listPositive[idxPos], listNegative[idxNeg]
                                    )
                                for arrNumber, arrIdx in enumerate(array):
                                    for entry in arrIdx:
                                        exponent = entry[0]*h + entry[1]*k + entry[2]*l
                                        if arrNumber == 0:
                                            fltStruct += pos * np.exp(-2j*np.pi*exponent)
                                        else:
                                            fltStruct += neg * np.exp(-2j*np.pi*exponent)
                            if abs(fltStruct) != 0:
                                fltM = h**2 + k**2 + l**2
                                if varFirst:
                                    fltFirst = fltM
                                    varFirst = False
                                if fltM not in listM[idxList] and np.sqrt(fltM / fltFirst) < 6:
                                    listM[idxList].append(fltM)
                                    listHKL.append([h, k, l, fltM, np.sqrt(fltM), number])
                if len(listHKL) < 25:
                    varSkip = True
                    break
            minimumFltM = 100
            for line in listHKL:
                if line[3] < minimumFltM:
                    minimumFltM = line[3]
            for line in listHKL:
                line[3] = line[3] / np.sqrt(minimumFltM)
                
            if varSkip:
                continue
            with open(
                    strOut, 'w'
                    ) as f:
                for row in listHKL:
                    for entry in row:
                        f.write('{0}\t'.format(entry))
                    f.write('\n')

In [4]:
# Two atoms but same base

In [5]:
# Load vector data

In [18]:
# SC
arrSC = np.array([np.genfromtxt('raw/vec_sc.dat')])
# BCC
arrBCC = np.genfromtxt('raw/vec_bcc.dat')
# FCC
arrFCC = np.genfromtxt('raw/vec_fcc.dat')
# Diamant
arrDia = np.genfromtxt('raw/vec_diamant.dat')
# Zinkblende
arrZinkA = np.genfromtxt('raw/vec_zinkblende_a.dat')
arrZinkB = np.genfromtxt('raw/vec_zinkblende_b.dat')
arrZink = np.array([arrZinkA, arrZinkB])
arrZinkInv = np.array([arrZinkB, arrZinkA])
arrZinkOne = np.append(arrZinkB, arrZinkA, axis=0)
# Steinsalz
arrSteinA = np.genfromtxt('raw/vec_steinsalz_a.dat')
arrSteinB = np.genfromtxt('raw/vec_steinsalz_b.dat')
arrStein = np.array([arrSteinA, arrSteinB])
arrSteinInv = np.array([arrSteinB, arrSteinA])
arrSteinOne = np.append(arrSteinB, arrSteinA, axis=0)
# Fluorid
arrFluoridA = np.genfromtxt('raw/vec_fluorid_a.dat')
arrFluoridB = np.genfromtxt('raw/vec_fluorid_b.dat')
arrFluorid = np.array([arrFluoridA, arrFluoridB])
arrFluoridInv = np.array([arrFluoridB, arrFluoridA])
arrFluoridOne = np.append(arrFluoridB, arrFluoridA, axis=0)
# Caesium Chlorid
arrCaesiumA = np.array([np.genfromtxt('raw/vec_caesium_a.dat')])
arrCaesiumB = np.array([np.genfromtxt('raw/vec_caesium_b.dat')])
arrCaesium = np.array([arrCaesiumA, arrCaesiumB])
arrCaesiumInv = np.array([arrCaesiumB, arrCaesiumA])
arrCaesiumOne = np.append(arrCaesiumB, arrCaesiumA, axis=0)

In [7]:
# Load ions

In [19]:
arrNegative = np.genfromtxt('raw/form_negative.dat')
arrPositive = np.genfromtxt('raw/form_positive.dat')

In [9]:
arrIons = np.append(
    arrNegative, arrPositive, axis=0
    )
arrCombined = np.append(
    np.array([[
            0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 
            0.30, 0.35, 0.40, 0.50, 0.60, 0.70
        ]]),
    arrIons, axis=0
)
listNames = [r'$\frac{\sin{\Theta}}{\lambda}\,/\,\si{1\per\angstrom}$' ,r'$F^-$', r'$Cl^-$', r'$J^-$', r'$K^+$', r'$Ca^{2+}$', r'$Cu^+$', r'$Cs^+$']

with open('build/tex/formfaktoren.tex', 'w') as f:
    f.write(
        table(
            listNames,
            arrCombined
        )
    )

In [10]:
# Lists for iteration

In [20]:
listArrOne = [
    arrSC, arrBCC, arrFCC, arrDia
]
listArrTwo = [
    arrStein, arrFluorid, arrZink, arrCaesium,
    arrSteinInv, arrFluoridInv, arrZinkInv, arrCaesiumInv
]
listArrNameOne = [
    'SC', 'BCC', 'FCC', 'Diamant', 
    'Steinsalz', 'Fluorid', 'Zinkblende', 'Caesiumstruktur'
]
listArrNameTwo = [
    'Steinsalz', 'Fluorid', 'Zinkblende', 'Caesiumstruktur',
    'Steinsalz', 'Fluorid', 'Zinkblende', 'Caesiumstruktur'
]
listNumber = [
    '0', '5', '10', '15', '20', '25', 
    '30', '35', '40', '50', '60', '70'
]
listNegative = [
    'Fluor', 'Chlor', 'Jod'
]
listPositive = [
    'Kalium', 'Calcium', 'Kupfer', 'Caesium'
]

In [12]:
# Calculate files

In [28]:
for array, name in zip(listArrOne, listArrNameOne):
    _one_atom(array, name)
       
for idx, array, name in zip(range(len(listArrTwo)), listArrTwo, listArrNameTwo):
    if idx >= len(listArrTwo) / 2:
        _two_atoms(array, name, arrPositive, arrNegative, listNumber, listPositive, listNegative, inv=True)
    else:
        _two_atoms(array, name, arrPositive, arrNegative, listNumber, listPositive, listNegative, inv=False)

In [14]:
#_two_atoms(listArrTwo[3], listArrNameTwo[3], arrPositive, arrNegative, listNumber, listPositive, listNegative, inv=False)